## Import libaries

In [ ]:
import os
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D

## Mount Dataset

In [ ]:
from azureml.core import Dataset, Workspace, Model
workspace = Workspace.from_config()

In [ ]:
dataset = Dataset.get_by_name(workspace, name='RGBDS-150K-test', version=2)
mount_ctx = dataset.mount()  
mount_ctx.start()
mount_path = mount_ctx.mount_point
mount_path

## Load CSV and Filter

In [ ]:
test_df = pd.read_csv('test_result_mobilenet.csv')
test_df['errors'] = test_df['predicted_height'] - test_df['height']
test_df["pose_data_2"] = test_df["pose_data"].apply(lambda x: eval(x))
test_df["pose_score"] = test_df["pose_data_2"].apply(lambda x: float(x["Pose Scores"]))
print(f"Number of artifacts in the test dataset: {len(test_df)}")

In [ ]:
filtered_test_df = test_df[
    (test_df['distance_to_child'] > 0.1) &
    (test_df['overlap'] > 90) &
    (test_df['pose_score'] > 0.8) & 
    (test_df['angle'].between(-60, -10))
]    

## Functions for viewing

In [ ]:
from depthmap_preprocess import replace_values_above_threshold_by_zero, fill_zeros_inpainting

In [ ]:
def load_data_from_file(file_path):
    """
    Load data from a file and process depth data.

    Parameters:
    - file_path (str): The relative path to the file.

    Returns:
    dict: A dictionary containing 'rgb', 'depth', 'clean_depth', and 'targets' data.
    """
    full_path = mount_path + '/' + file_path
    with open(full_path, "rb") as f:
        [depthmap_data, rgb_data, segmap_data], targets_lst = pickle.load(f)

    clean_depth_data = fill_zeros_inpainting(replace_values_above_threshold_by_zero(depthmap_data, 3.0))
    
    return {
        'rgb': rgb_data,
        'depth': np.squeeze(depthmap_data),
        'clean_depth': np.squeeze(clean_depth_data),
        'targets': targets_lst
    }

In [ ]:
def display_image_grid(file_paths, image_width=3, image_height=4, errors=None):
    """
    Display a grid of images and associated details.

    Parameters:
    - file_paths (list): List of file paths to load data from.
    - image_width (int): Width of each image in inches.
    - image_height (int): Height of each image in inches.
    - errors (list): List of error values for each image (default is None).

    Returns:
    None
    """
    num_images = len(file_paths)
    if errors is None:
        errors = [np.nan] * num_images
    
    # Set the number of images per grid
    images_per_grid = min(num_images, 9)
    # Calculate the number of grids needed
    num_grids = (num_images + images_per_grid - 1) // images_per_grid
    
    # Set up the matplotlib figure and axes for each grid
    for grid_index in range(num_grids):
        start_index = grid_index * images_per_grid
        end_index = min((grid_index + 1) * images_per_grid, num_images)
        
        grid_fig, grid_axs = plt.subplots(5, images_per_grid,  
                                          figsize=(image_width * images_per_grid, image_height * 5), 
                                          #subplot_kw={'xticks': [], 'yticks': []},
                                          gridspec_kw={'hspace': 0.0, 'wspace': 0.0})
        
        # Ensure grid_axs is iterable by making it a list if there's only one subplot
        grid_axs = grid_axs.T if images_per_grid > 1 else [grid_axs]
        
        for i, (ax, file_path, err) in enumerate(zip(grid_axs, file_paths[start_index:end_index], errors[start_index:end_index])):
            data_dict = load_data_from_file(file_path)

            # Display RGB data in the first row
            rgb_data = data_dict['rgb']
            ax[1].imshow(rgb_data)
            ax[1].axis('off')

            depth_data = data_dict['depth']
            ax[2].imshow(depth_data, cmap='jet', vmin=0, vmax=3)
            ax[2].axis('off')
            
            ax[4].hist(depth_data.flatten(), bins=50, range=(0, 7), color='blue', alpha=0.7, histtype='barstacked')
            ax[4].set_ylim([0, 15000])
            ax[4].tick_params(axis='x', which='both', bottom=True, top=False, labelbottom=True) 
            ax[4].tick_params(axis='y', which='both', left=True, right=False, labelleft=True)   

            # Display Clean Depth data in the fourth row
            clean_depth_data = data_dict['clean_depth']
            ax[3].imshow(clean_depth_data, cmap='jet', vmin=0, vmax=3)
            ax[3].axis('off')

            ax[0].axis('off')
            non_zero_depth_values = depth_data[depth_data > 0]
            count_above_threshold = np.count_nonzero(non_zero_depth_values > 3.0)
            max_depth = np.max(non_zero_depth_values)
            min_depth = np.min(non_zero_depth_values)
            avg_depth = np.mean(non_zero_depth_values)


            overlap = data_dict['targets']['overlap']
            
            distance_to_child = data_dict['targets']['distance_to_child']
            angle = data_dict['targets']['angle']
            pose = data_dict['targets']['pose_data'].get('Pose Scores', 'N/A')
            pose_score_float = float(pose) if pose != 'N/A' else None

            details_text = f"Error: {err:.2f}\nMaxDepth: {max_depth:.2f}\nMinDepth: {min_depth:.2f}\nAvgDepth: {avg_depth:.2f}\nCount > 3.0: {count_above_threshold}\nDistance: {distance_to_child:.2f}\nAngle: {angle:.2f}\nOverlap: {overlap:.2f}\nPoseScore: {pose_score_float:.2f}"

            ax[0].text(0.5, 0.5, details_text, ha='center', va='center', fontsize=18)

        plt.tight_layout()
        plt.show()

        if num_grids > 1:
            print(f"Displaying grid {grid_index + 1} of {num_grids}")

In [ ]:
def plot_parameter_vs_height(ax, parameter_list, height_list, unique_height, label, min_param=None, max_param=None, min_height=None, max_height=None):
    """
    Plot a scatter plot of a parameter versus predicted and actual heights.

    Parameters:
    - ax (matplotlib.axes._axes.Axes): Matplotlib Axes object.
    - parameter_list (list): List of parameter values.
    - height_list (list): List of predicted heights.
    - unique_height (float): Actual height value.
    - label (str): Label for the x-axis.
    - min_param (float): Minimum value for the x-axis (default is None).
    - max_param (float): Maximum value for the x-axis (default is None).
    - min_height (float): Minimum value for the y-axis (default is None).
    - max_height (float): Maximum value for the y-axis (default is None).

    Returns:
    None
    """
    
    # Plotting the predicted height
    ax.scatter(parameter_list, height_list, label='P-Height', marker='o', color='red')

    # Scatter plot for actual heights
    ax.axhline(y=unique_height, color='green', linestyle='solid', label=f'A-Height: {unique_height:.2f}')

    # Mark mean, median, q25, and q75 of the parameter list
    mean_parameter = np.mean(parameter_list)
    median_parameter = np.median(parameter_list)
    q25_parameter = np.percentile(parameter_list, 25)
    q75_parameter = np.percentile(parameter_list, 75)

    ax.axvline(x=mean_parameter, color='purple', linestyle=':', label=f'Mean {label}: {mean_parameter:.2f}')
    ax.axvline(x=median_parameter, color='orange', linestyle=':', label=f'Median {label}: {median_parameter:.2f}')
    ax.axvline(x=q25_parameter, color='cyan', linestyle=':', label=f'Q25 {label}: {q25_parameter:.2f}')
    ax.axvline(x=q75_parameter, color='magenta', linestyle=':', label=f'Q75 {label}: {q75_parameter:.2f}')

    # Mark mean and median of predicted heights
    mean_predicted_height = np.mean(height_list)
    median_predicted_height = np.median(height_list)

    ax.axhline(y=mean_predicted_height, color='blue', linestyle='-', label=f'Mean P-Height: {mean_predicted_height:.2f}')
    ax.axhline(y=median_predicted_height, color='yellow', linestyle='-', label=f'Median P-Height: {median_predicted_height:.2f}')

    ax.set_title(f'{label} vs. Predicted and Actual Height')
    ax.set_xlabel(label)
    ax.set_ylabel('Height')

    # Adjust the position of the legend
    ax.legend(loc='upper left', bbox_to_anchor=(0, -0.2))

    ax.grid(True)

    # Set axis limits if provided
    if min_param is not None and max_param is not None:
        ax.set_xlim([min_param, max_param])
    if min_height is not None and max_height is not None:
        ax.set_ylim([min_height, max_height])


In [ ]:
def plot_all_parameters_graph(angle_list, distance_to_child_list, pose_score_list, overlap_list, predicted_height_list, unique_heights):
        """
    Plot multiple parameter versus predicted and actual height graphs in a single figure.

    Parameters:
    - angle_list (list): List of angle values.
    - distance_to_child_list (list): List of distance to child values.
    - pose_score_list (list): List of pose score values.
    - overlap_list (list): List of overlap values.
    - predicted_height_list (list): List of predicted heights.
    - unique_heights (float): Actual height value.

    Returns:
    None
    """
    fig, axs = plt.subplots(1, 4, figsize=(20, 5))
    plot_parameter_vs_height(axs[0], angle_list, predicted_height_list, unique_heights, 'Angle')
    plot_parameter_vs_height(axs[1], distance_to_child_list, predicted_height_list, unique_heights, 'Distance to Child')
    plot_parameter_vs_height(axs[2], pose_score_list, predicted_height_list, unique_heights, 'Pose Score')
    plot_parameter_vs_height(axs[3], overlap_list, predicted_height_list, unique_heights, 'Overlap')
    plt.subplots_adjust(wspace=0.4)
    plt.show()

In [ ]:
def plot_3d(height_list, angle_list, distance_list, unique_height, actual_height_color='green'):
    """
    Plot a 3D scatter plot of predicted height, angle, and distance to child.

    Parameters:
    - height_list (list): List of predicted heights.
    - angle_list (list): List of angles.
    - distance_list (list): List of distances to child.
    - unique_height (float): Actual height value (for a horizontal plane).
    - actual_height_color (str): Color for the scatter plot of actual height.

    Returns:
    None
    """
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(angle_list, distance_list, height_list, label='Predicted Height', color='red', marker='o')

    ax.scatter(angle_list, distance_list, unique_height, label='Actual Height', color=actual_height_color, marker='o')

    ax.set_xlabel('Angle')
    ax.set_ylabel('Distance to Child')
    ax.set_zlabel('Height')
    ax.set_title('3D Plot of Height, Angle, and Distance')

    ax.legend()
    plt.show()


In [ ]:
def plot_contour(height_list, angle_list, distance_list, title='Contour Plot of Height', color_map='viridis'):
    """
    Plots a contour plot for the given height, angle, and distance data.

    :param height_list: List or array of height values.
    :param angle_list: List or array of angle values.
    :param distance_list: List or array of distance values.
    :param title: Title for the plot.
    :param color_map: Color map for the contour plot.
    """

    xi = np.linspace(min(angle_list), max(angle_list), 100)
    yi = np.linspace(min(distance_list), max(distance_list), 100)
    xi, yi = np.meshgrid(xi, yi)

    zi = griddata((angle_list, distance_list), height_list, (xi, yi), method='cubic')

    if np.isnan(zi).any():
        zi = np.nan_to_num(zi)

    if not np.all(np.isnan(zi)):
        plt.figure(figsize=(10, 8))
        contour = plt.contourf(xi, yi, zi, levels=np.linspace(np.nanmin(zi), np.nanmax(zi), 100), cmap=color_map)
        plt.colorbar(contour)

        plt.xlabel('Angle')
        plt.ylabel('Distance to Child')
        plt.title(title)

        plt.show()
    else:
        print("The interpolated data is not valid for contour plotting.")

In [ ]:
def plot_3d_surface_with_actuals(predicted_height_list, actual_height_list, angle_list, distance_list, title='3D Surface Plot of Predicted vs Actual Height'):
    """
    Plots a 3D surface plot for the given predicted height and overlays actual height data points, with angle and distance data.

    :param predicted_height_list: List or array of predicted height values.
    :param actual_height_list: List or array of actual height values.
    :param angle_list: List or array of angle values.
    :param distance_list: List or array of distance values.
    :param title: Title for the plot.
    """
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    xi = np.linspace(min(angle_list), max(angle_list), len(np.unique(angle_list)))
    yi = np.linspace(min(distance_list), max(distance_list), len(np.unique(distance_list)))
    xi, yi = np.meshgrid(xi, yi)

    zi = griddata((angle_list, distance_list), predicted_height_list, (xi, yi), method='linear')

    mask = ~np.isnan(zi)
    xi, yi, zi = xi[mask], yi[mask], zi[mask]

    surface = ax.plot_trisurf(xi, yi, zi, cmap='viridis', alpha=0.7)

    ax.scatter(angle_list, distance_list, actual_height_list, color='red', label='Actual Height')

    ax.set_xlabel('Angle')
    ax.set_ylabel('Distance to Child')
    ax.set_zlabel('Predicted Height')
    ax.set_title(title)

    ax.legend()

    fig.colorbar(surface, shrink=0.5, aspect=5)

    plt.show()

## View filtered images in Grid (set of artifacts)

In [ ]:
single_scan = filtered_test_df.tail(9).copy()
single_path = single_scan['pickle_file_path'].tolist()
errors_list = single_scan['errors'].tolist()
scan_id_list = single_scan['scan_id'].tolist()
print(scan_id_list)
display_image_grid(single_path, errors= errors_list)

## View all images at child level and analyze

In [ ]:
unique_person_ids_filtered = filtered_test_df['person_id'].unique()
len(unique_person_ids_filtered)

In [ ]:
pid = unique_person_ids_filtered[50] #Change the person_id here
single_child = test_df[test_df['person_id']==pid]
unique_scan_ids = single_child['scan_id'].unique()

In [ ]:
print(f"\nperson_id: {pid}")
for scan_id in unique_scan_ids:
    print(f"scan_id: {scan_id}")
    single_scan = test_df[test_df['scan_id'] == scan_id]
    unique_heights = single_scan['height'].unique()[0]
    print(f"Height: {unique_heights}")
    single_path = single_scan['pickle_file_path'].tolist()
    errors_list = single_scan['errors'].tolist()
    angle_list = single_scan['angle'].tolist()
    distance_to_child_list = single_scan['distance_to_child'].tolist()
    predicted_height_list = single_scan['predicted_height'].tolist()
    pose_score_list = single_scan['pose_score'].tolist()
    overlap_list = single_scan['overlap'].tolist()
    height_list = single_scan['height'].tolist()
    
    display_image_grid(single_path, errors= errors_list)

    plot_all_parameters_graph(angle_list, distance_to_child_list, pose_score_list, overlap_list, predicted_height_list, unique_heights)

    #plot_3d(predicted_height_list, angle_list, distance_to_child_list, height_list, actual_height_color='green')
    
    #plot_3d_surface_with_actuals(predicted_height_list, height_list, angle_list, distance_to_child_list)

## Unmount Dataset

In [ ]:
mount_ctx.stop()